In [7]:
from datasets import load_dataset
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()

# Load dataset from Hugging Face Hub
dataset = load_dataset("fitlemon/rag-labor-codex-dataset", token=os.getenv("HF_KEY"))

In [8]:
# Импорт для перевода
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
import os

def translate_uz_ru(message: str) -> str:
    """
    Переводит текст с узбекского на русский.
    """
    prompt = ChatPromptTemplate.from_messages([
        (
            "system",
            "You are a helpful assistant that translates {input_language} to {output_language}. The subject of Text is Labor codex. Example input: tatil. Output: отпуск."
        ),
        ("human", "{input}")
    ])
    llm = ChatOpenAI(
        api_key=os.environ["OPENAI_API_KEY"],
        model="gpt-4o-mini-2024-07-18"
    )
    # Цепочка для перевода
    chain = prompt | llm
    response = chain.invoke({
        "input_language": "Uzbek",
        "output_language": "Russian",
        "input": message,  # Передаём сообщение как строку, а не множество!
    })
    
    return response.content

In [9]:
# sample
sample = dataset["train"][0]['chunk']

In [5]:
# Переводим текст
translated = translate_uz_ru(sample)

C:\Users\Davron\AppData\Local\Temp\ipykernel_22140\272192548.py:17: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [6]:
translated

'Работы, выполняемые подрядчиком с участием (помощью) членов его семьи, могут осуществляться без согласия работодателя. При этом между членами семьи подрядчика и работодателем не возникают индивидуальные трудовые отношения. Подрядчик несет ответственность перед работодателем за качество работ (услуг), выполняемых членами его семьи, которые ему помогают.'

In [15]:
from tqdm import tqdm

# get unique chunks and translate them
chunks = set()

for item in dataset["train"]:
    chunks.add(item['chunk'])

for item in dataset['train']:
    chunks.add(item['chunk'])

# save dict with translations
translations = {}

for chunk in tqdm(chunks):
    translations[chunk] = translate_uz_ru(chunk)

100%|██████████| 662/662 [44:52<00:00,  4.07s/it]  


In [19]:
# save to pickle
import pickle

with open('translations.pkl', 'wb') as f:
    pickle.dump(translations, f)